# Proyecto: Funciones definidas por el usuario (User Defined Functions: UDF)

En la presente práctica veremos cómo:

* Crear RDDs a partir archivos .csv

* Crear Data Frames a partir de RDDs

* Construcción y Aplicación de funciones UDF (Veremos como convertir el tipo de dato de una columna y asignar valor 'Null' a campos literalmente vacíos)

In [16]:
# libreria para crear punto de conexión:
from pyspark import SparkContext

# Cargamos libreria para crear Data Frames:
from pyspark.sql import SQLContext

#from pyspark.sql import SparkSession

# Cargamos librerias para crear el schema del DataFrame
from pyspark.sql.types import StructType, StructField

# Cargamos los tipos de datos que usaremos para crear las columnas de los dataframes:
from pyspark.sql.types import  IntegerType, StringType, FloatType

#from pyspark.sql.types import Row

# Cargamos librería para poder crear funciones UDF:
from pyspark.sql.functions import udf

In [2]:
# Creamos función para eliminar encabezados en RDDs:
def eliminaEncabezado(indice , interador):
    return iter( list(interador)[1:] )

In [3]:
# Creamos el punto de conexió a Spark (que se ejecutará en mi máquina 'local'):
spark = SparkContext(master='local', appName='UDF_Functions')

In [4]:
# Creamos el contexto para SQL:
sqlContext = SQLContext(spark)

/home/marck/.local/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
!ls Data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [6]:
# Vemos las primeras 5 líneas del archivo 'deportistaError.csv'

!head -n 5 Data/deportistaError.csv

# Veremos que el archivo ya tiene un encabezado y hay algunos campos faltantes

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,,,273
4,Edgar Lindenau Aabye,1,34,,,278


In [7]:
# Creamos la ruta de acceso a los archivos:
path = './Data/'

In [8]:
# Número de registros a visualizar:
N = 5

## Creamos RDD a partir de un archivo .csv:

In [9]:
# Creamos RDDs con el archivo 'deportistaError.csv'
# .map(lambda l : l.split(',')) <-- Le asigan formato al contenido de archivos .csv

deportistaErrorRDD = spark.textFile(path+'deportistaError.csv').map(lambda l : l.split(','))

In [11]:
# Obtenemos los primeros N registros:
deportistaErrorRDD.take(N)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '', '', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '', '', '278']]

In [12]:
# Eliminamos encabezado del RDD :
deportistaErrorRDD = deportistaErrorRDD.mapPartitionsWithIndex(eliminaEncabezado)

# Vemos los primeros N registros del RDD:
deportistaErrorRDD.take(N)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '', '', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '', '', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

## Pasamos RDD a un Data Frame:

In [13]:
# Creación de un Data Frame a partir de un RDD:

# En este caso (intencionalmente) no asignamos tipo de dato a las columnas del Data Frame:
deportistaErrorRDD = \
    deportistaErrorRDD.map(lambda x: ( x[0], x[1],  x[2], x[3], x[4], x[5], x[6] ) )


# A continuación creamos el schema, es decir, la estructura (columnas) del Data Frame donde pondremos los datos del RDD:
schema = StructType([
    StructField('deportista_id', StringType(), False ),
    StructField('nombre', StringType(), False ),
    StructField('genero', StringType(), False ),
    StructField('edad', StringType(), False ),
    StructField('altura', StringType(), False ),
    StructField('peso', StringType(), False ),
    StructField('equipo_id', StringType(), False )
    ])

# StructField <-- instrucción para crear una columna, se debe especificar el tipo de dato que almacenará
# En caso de que el campo pueda ser nulo ponemos 'True' de lo contrario ponemos 'False'

# Creamos el DataFrame a partir del RDD:
deportistaErrorDF = sqlContext.createDataFrame(deportistaErrorRDD, schema)

In [15]:
# Vemos los primero registros del Data Frame:
deportistaErrorDF.show()

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|  80|      199|
|            2|            A Lamusi|     1|  23|   170|  60|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|      |    |      273|
|            4|Edgar Lindenau Aabye|     1|  34|      |    |      278|
|            5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|            6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|            7|        John Aalberg|     1|  31|   183|  72|     1096|
|            8|"Cornelia ""Cor""...|     2|  18|   168|    |      705|
|            9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
|           10|"Einar Ferdinand ...|     1|  26|      |    |      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
|     

Notemos que existen campos vacíos (literlmente vacíos) y debemos asignarles el valor 'None'

## Creamos función que usaremos como 'udf'

In [18]:
# Función que convierte al tipo de dato 'integer' y
# si el campo es vacío, asigan el valor 'null -> None'
def conversionEnteros(valor):
    return int(valor) if len(valor) > 0 else None

# Declaramos función conmo udf:
conversionEnteros_udf = udf(lambda z: conversionEnteros(z) , IntegerType() )

# Damos de alta la función udf:
sqlContext.udf.register('conversionEnteros_udf', conversionEnteros_udf)

<function __main__.<lambda>(z)>

## Aplicamos la función udf:

In [22]:
# Imprimimos el schema antes de aplicar la función udf:
deportistaErrorDF.printSchema()

root
 |-- deportista_id: string (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: string (nullable = false)
 |-- edad: string (nullable = false)
 |-- altura: string (nullable = false)
 |-- peso: string (nullable = false)
 |-- equipo_id: string (nullable = false)



In [30]:
# Aplicamos la función udf a la columna 'altura'
deportistaErrorDF.select( conversionEnteros_udf('altura').alias('alturaUDF') ).show()

+---------+
|alturaUDF|
+---------+
|      180|
|      170|
|     null|
|     null|
|      185|
|      188|
|      183|
|      168|
|      186|
|     null|
|      182|
|      172|
|      159|
|      171|
|     null|
|      184|
|      175|
|      189|
|     null|
|      176|
+---------+
only showing top 20 rows



In [31]:
# Cerramos sesión para liberar memoria:
spark.stop()